## RL agent Q-learning for TicTacToe env

The [Tic-Tac-Toe](https://github.com/MauroLuzzatto/OpenAI-Gym-TicTacToe-Environment) is a simple game environment that allows to train reinforcement learning agents. These notebook contains an implemetation of Q-learning with epsilon-greedy strategy for TicTacToe env.

In [291]:
# load the python modules
import time
import sys
import warnings

import gym
import numpy as np
from tqdm import tqdm
import gym_TicTacToe

from src.qagent import QLearningAgent
from src.play_tictactoe import play_tictactoe, play_tictactoe_with_random

from src.utils import (
    create_state_dictionary,
    reshape_state,
    save_qtable,
    load_qtable
)

# ignore warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [292]:
class Player:
    def __init__(self, color, episodes: int):
        self.color = color
        self.reward_array = np.zeros(episodes)
        self.reset_reward()
        self.name = f"Player {color}"

    def reset_reward(self):
        self.reward = 0

In [293]:

# initialize the tictactoe environment
env = gym.envs.make("TTT-v0", small=-1, large=10)

In [294]:
state_dict = create_state_dictionary()
state_size = len(state_dict.keys())
action_size = env.action_space.n

Number of legal states: 12092


In [295]:
# set training parameters
episodes = 960_000
max_steps = 9

In [296]:
exploration_parameters = {
    "max_epsilon": 1.0,
    "min_epsilon": 0.0,
    "decay_rate": 0.00001,
}

In [297]:
qagent = QLearningAgent(exploration_parameters, state_size, action_size, learning_rate=0.1, gamma=0.99)

In [298]:
def check_for_potential_lose(state, color: int) -> bool:
        """check if after agent's move there is a lose position

        Args:
            color (int): of the player's enemy

        Returns:
            bool: indicating if this was a crucial move
        """
        state_check = np.copy(state)
        lose = False
        col = np.array([1,2])
        #enemy color
        enemy_color = color
        player_color = col[col != enemy_color][0]
        state_check[state_check == player_color] = -1
        state_check[state_check == enemy_color] = 1
        state_check = state_check.reshape(3,3)
        for ii in range(3):
            if (
                # check columns
                np.sum(state_check[:, ii]) == 2
                # check rows
                or np.sum(state_check[ii, :]) == 2
                # check diagonal
                or np.sum([state_check[0, 0], state_check[1, 1], state_check[2, 2]])
                == 2
                or np.sum([state_check[0, 2], state_check[1, 1], state_check[2, 0]])
                == 2
            ):
                lose = True
                break
        return lose

In [299]:
state = np.array([0,1,2,
                  0,1,2,
                  2,0,1])

check_for_potential_lose(state, color=1)

True

In [300]:
def play(qagent:QLearningAgent, player_color, state: int, action_space: np.array, last_turn: bool) -> tuple:
    action = qagent.get_action(state, action_space)

    # remove action from the action space
    action_space = action_space[action_space != action]

    new_state, reward, done, _ = env.step((action, player_color))

    col = np.array([1,2])
    # if done:
    #     print(new)
    #     reward -= 5
    if (done == False):
        if check_for_potential_lose(new_state, col[col != player_color][0]):
            reward += 5

    # if (last_turn == True) and (done == False):
    #     reward += 9     # Reward for draw

    # elif (last_turn == True) and (done == True):
    #     reward += 10
        # print(f"New_state:{new_state}, Reward:{reward}, Done:{done}")
    #TODO: maybe should change a marker after this agent turn 
    new_state = np.append(new_state, player_color)
    new_state = state_dict[reshape_state(new_state)] 

    qagent.qtable[state, action] = qagent.update_qtable(
        state, new_state, action, reward, done
    )
    # new state
    state = new_state
    return state, action_space, done

In [301]:
def play_random(qagent:QLearningAgent, player_color, state: int, action_space: np.array) -> tuple:
    action = np.random.choice(action_space)
    action_space = action_space[action_space != action]
    new_state, reward, done, _ = env.step((action, player_color))
    new_state = np.append(new_state, player_color)
    new_state = state_dict[reshape_state(new_state)]
    state = new_state
    return state, action_space, done

In [302]:
visited_states = np.zeros((state_size, 1))

In [303]:
lear_rate = 0.8
gamma = 0.9
qagent = QLearningAgent(exploration_parameters, state_size, action_size, learning_rate=lear_rate, gamma=gamma)

In [304]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
import random

start_time = time.time()

player_1 = Player(color=1, episodes=episodes)
player_2 = Player(color=2, episodes=episodes)

track_progress = np.zeros(episodes)

win_history = []

rewards = []
# lr = 0.4, gamma = 0.9, winrate = 0.64
# Learning rate: 0.5, Win rate: 0.72, Gamma: 0.9
# Learning rate: 0.6, Win rate: 0.5, Gamma: 0.9
# Learning rate: 0.7, Win rate: 0.66, Gamma: 0.9
# Learning rate: 0.65, Win rate: 0.76, Gamma: 0.8

# best 
# Learning rate: 0.8, Win rate: 0.8, Gamma: 0.9

# qagent_old = qagent
for episode in tqdm(range(episodes)):
    last_turn = False
    action_space = np.arange(9)
    player_1.reset_reward()
    player_2.reset_reward()

    # randomly change the order players
    start = np.random.choice([1,2])

    state, _ = env.reset()
    state = np.append(state, start)
    state = state_dict[reshape_state(state)]
    # if episode % 10_000 == 0:
    #     save_qtable(qagent.qtable, 'tables', "q_table_old")

    for _step in range(start, max_steps + start):
        if _step == max_steps + start - 1:
            last_turn = True
        # change a turn
        if _step % 2 == 0:
            # state, action_space, done = play_random(qagent, player_1.color, state, action_space)
            #qagent_old.qtable = load_qtable('tables', "q_table_old")
            state, action_space, done = play(qagent, player_1.color, state, action_space, last_turn)
        else:
            state, action_space, done = play(qagent, player_2.color, state, action_space, last_turn)
        visited_states[state] += 1
        if done == True:
            break

    # reduce epsilon for exporation-exploitation tradeoff
    qagent.update_epsilon(episode)

    #cur_win_rate, reward = play_tictactoe_with_random(env, qagent.qtable, state_dict, num_test_games=100)

    #check how good is agent
    if episode % 25_000 == 0:
        num_games = 50
        cur_win_rate, reward = play_tictactoe_with_random(env, qagent.qtable, state_dict, num_test_games=num_games)
        win_history.append(sum(cur_win_rate)/num_games)
        print("WinRate:", sum(cur_win_rate)/num_games)
        # rewards.append(reward)
        # clear_output(True)
        # # plt.title('eps = {:e}, mean reward = {:.1f}'.format(agent.epsilon, np.mean(rewards[-10:])))
        # plt.plot(rewards)
        # plt.show()
    if episode % 25_000 == 0:

        sum_q_table = np.sum(qagent.qtable)
        time_passed = round((time.time() - start_time) / 60.0, 2)

        print(
            f"episode: {episode}, \
            epsilon: {round(qagent.epsilon, 2)}, \
            sum q-table: {sum_q_table}, \
            elapsed time [min]: {time_passed},  \
            done [%]: {episode / episodes * 100} \
            "
        )


  0%|          | 145/960000 [00:00<21:19, 749.98it/s]

WinRate: 0.32
episode: 0,             epsilon: 1.0,             sum q-table: 35.28769055589007,             elapsed time [min]: 0.0,              done [%]: 0.0             


  3%|▎         | 25110/960000 [00:25<17:02, 914.34it/s] 

WinRate: 0.6
episode: 25000,             epsilon: 0.78,             sum q-table: 322947.1921351756,             elapsed time [min]: 0.42,              done [%]: 2.604166666666667             


  5%|▌         | 50166/960000 [00:51<16:36, 912.71it/s] 

...

 96%|█████████▋| 925211/960000 [19:51<00:48, 721.10it/s]

WinRate: 0.92
episode: 925000,             epsilon: 0.0,             sum q-table: 407696.23080211226,             elapsed time [min]: 19.85,              done [%]: 96.35416666666666             


 99%|█████████▉| 950199/960000 [20:24<00:13, 721.22it/s]

WinRate: 0.92
episode: 950000,             epsilon: 0.0,             sum q-table: 407696.23080211226,             elapsed time [min]: 20.4,              done [%]: 98.95833333333334             


100%|██████████| 960000/960000 [20:37<00:00, 775.79it/s]


In [305]:
visited_states.shape[0]
print("Percent:",100*np.sum(visited_states > 0)/visited_states.shape[0])

Percent: 90.58881905391995


In [313]:
num_games = 1000
cur_win_rate, _ = play_tictactoe_with_random(env, qagent.qtable, state_dict, num_test_games=num_games)
win_history.append(sum(cur_win_rate)/num_games)
print("WinRate:", sum(cur_win_rate)/num_games)

WinRate: 0.858


In [314]:
qtable = qagent.qtable
save_qtable(qtable, 'tables', "q_table_best2")

q_table_best2.npy saved!


In [6]:
qtable = load_qtable('tables', "q_table_best")

In [324]:
#check how correct is q-table

state = np.random.choice(np.arange(env.observation_space.n))
# state_dict[state]
print(state)

key = list(filter(lambda x: state_dict[x] == state, state_dict))[0]
print(np.array(key[:-1]).reshape(3,3))
print("Turn was:", key[-1])
print(np.round(qagent.qtable[state].reshape(3,3),1))


1793
[[2 1 2]
 [1 0 2]
 [2 1 0]]
Turn was: 2
[[ 0.   0.   0. ]
 [ 0.   9.   0. ]
 [ 0.   0.  12.1]]


In [10]:
play_tictactoe(env, qtable, state_dict, num_test_games=1)

Agent beginns
--------------------
╒═══╤═══╤═══╕
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛
--------------------
Move Human
Action: 4
╒═══╤═══╤═══╕
│ - │ - │ - │
├───┼───┼───┤
│ - │ X │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛
-1


--------------------
move Agent
Action: 0
╒═══╤═══╤═══╕
│ O │ - │ - │
├───┼───┼───┤
│ - │ X │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛


--------------------
Move Human
Action: 1
╒═══╤═══╤═══╕
│ O │ X │ - │
├───┼───┼───┤
│ - │ X │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛
-1


--------------------
move Agent
Action: 6
╒═══╤═══╤═══╕
│ O │ X │ - │
├───┼───┼───┤
│ - │ X │ - │
├───┼───┼───┤
│ O │ - │ - │
╘═══╧═══╧═══╛


--------------------
Move Human
Action: 7
╒═══╤═══╤═══╕
│ O │ X │ - │
├───┼───┼───┤
│ - │ X │ - │
├───┼───┼───┤
│ O │ X │ - │
╘═══╧═══╧═══╛
-1


--------------------
move Agent
Action: 3
╒═══╤═══╤═══╕
│ O │ X │ - │
├───┼───┼───┤
│ O │ X │ - │
├───┼───┼───┤
│ O │ X │ - │
╘═══╧═══╧═══╛
********